[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/schwartz-cnl/Computational-Neuroscience-Class/blob/main/Image%20Statistics/ImageStatisticsTutorial_forClass2025_part1.ipynb)

In [ ]:
# Image statistics tutorial; part 1
# PCA and ICA on images
# OS 2025. Original in Matlab by OS used in Berkeley summer course.

import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic_2d
from scipy.stats import rayleigh
import sys
import os

In [ ]:
# load an image
from tensorflow.keras.preprocessing import image
!wget https://github.com/schwartz-cnl/Computational-Neuroscience-Class/blob/main/Convolutional%20Neural%20Network/ILSVRC2012_test_00026783.JPEG?raw=true -O ILSVRC2012_test_00026783.JPEG
import numpy as np
img_path = 'ILSVRC2012_test_00026783.JPEG'
img = image.load_img(img_path, target_size=(224, 224))
import matplotlib.pyplot as plt
plt.imshow(img)


In [ ]:
# collect image patches so that we can run PCA and ICA on them
# PCA and ICA on image patches
import numpy as np
import matplotlib.pyplot as plt

def get_patches(im, patch_dim, num_patches):
    # Pick random starting x and y positions in the image num_patches times, and take patch
    # as a column vector.

    patches = np.zeros((patch_dim**2, num_patches))
    for i in range(num_patches):
        thex, they = -1, -1
        while thex < 0 or thex > (im.shape[1] - patch_dim):
            thex = int(np.round(np.random.rand() * im.shape[1]))
        while they < 0 or they > (im.shape[0] - patch_dim):
            they = int(np.round(np.random.rand() * im.shape[0]))
        the_patch = im[they:they + patch_dim, thex:thex + patch_dim]
        patches[:, i] = the_patch.flatten()
    return patches

# Main part of the code
patch_dim = 12
num_patches = 10000
img = image.load_img(img_path, target_size=(224, 224));
im = image.img_to_array(img);
print(im.shape)
im = im[:,:,0];   # take just one color dimension (i.e., grayscale)
print(im.shape)
sig = get_patches(im, patch_dim, num_patches)
print(sig.shape)


In [ ]:
# Show some random patches

for i in range(25):
  plt.subplot(5,5,i+1)
  plt.imshow(sig[:, i].reshape(patch_dim, patch_dim), cmap='gray')

In [ ]:
# PCA
numsigs = sig.shape[1]
cov_matrix = np.dot(sig, sig.T) / numsigs
D, V = np.linalg.eigh(cov_matrix)
D = D[::-1]  # Reverse the eigenvalues
V = V[:, ::-1]  # Reverse the eigenvectors
mat = V
newsig = np.dot(mat.T, sig)
print(D.shape)
non_zero_ind = np.where(np.abs(D) > np.finfo(float).eps)[0]
print(non_zero_ind.shape)

# Whiten
whitener = np.diag(1.0 / np.sqrt(D[non_zero_ind]))
mat = np.dot(whitener, V[:, non_zero_ind].T)
newsig = np.dot(mat, sig)

# Plot the eigenvalues
plt.plot(D,'o')
plt.title("Eigenvalues")
plt.show()



In [ ]:
# Since first eigenvalue one is extremely high (why?), look at the spread of the remaining
plt.plot(D[1:],'o')
plt.title("Eigenvalues")
plt.show()


In [ ]:
# Display one of the principle component
# Choose different principle components by changing pc_index
# to see how the PCA feature changes
# (how does the PCA feature change for larger indices?)
pc_index = 5
#pc_index = 45
plt.imshow(mat[pc_index, :].reshape(patch_dim, patch_dim))

# Display one of the principle components
plt.imshow(mat[pc_index, :].reshape(patch_dim, patch_dim), cmap='gray')
plt.title(f"PCA feature {pc_index}")
plt.axis('off')
plt.show()

In [ ]:
# We are also interested in the transform:
# sig = mat_inverse * newsig
# This is a transform from the output of pca back to the
# original image patches. The matrix mat_inverse is the
# inverse of mat. The columns of the matrix mat_inverse
# are known as basis functions. The input is a linear
# combination of the columns of mat_inverse.
# To view the basis functions, we need to view columns of mat_inverse
# Compute the inverse of the PCA transformation matrix
mat_inverse = np.linalg.inv(mat)

# Select one of the basis functions (column of mat_inverse)
pca_basis_index = 5
pca_basis_function = mat_inverse[:, pca_basis_index].reshape(patch_dim, patch_dim)

# Display the selected PCA basis function
plt.imshow(pca_basis_function, cmap='gray')
plt.title(f"PCA Basis Function {pca_basis_index}")
plt.axis('off')
plt.show()

In [ ]:
# ICA
# We want to see that ICA on images results in "edge" filters
from sklearn.decomposition import FastICA

ica = FastICA(n_components=patch_dim**2, random_state=0)
ica_sources = ica.fit_transform(sig.T)  # Shape is (num_patches, components)
print(ica_sources.shape)
ica_basis = ica.mixing_.T  # Shape is (components, patch_dim^2)
print(ica_basis.shape)


In [ ]:
# Select a random basis function to display
# Do these features have an order like we saw for the PCA?
random_index = np.random.randint(0, patch_dim**2)
random_basis_function = ica_basis[random_index, :].reshape(patch_dim, patch_dim)

# Display the selected ICA basis function
plt.imshow(random_basis_function, cmap='gray')
plt.title(f"ICA Basis Function {random_index}")
plt.axis('off')
plt.show()